In [1]:
import time
import sqlite3
import requests
import datetime

import pandas as pd
import shioaji as sj
import re
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar

import mplfinance as mpf


In [ ]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji()
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),  
          contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

In [5]:
import time
def get_ticks(connection, api, date, codes = str, is_Futures = False):

    dates = pd.to_datetime(date)
    tw_calendar = get_calendar('XTAI')

    #name tickers and correct time stamp

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = dates.date()+datetime.timedelta(days=1)
        
        else:
            date = dates.date()

    else:
        code = codes
        date = dates.date()
        

    try:    
        sql = "SELECT * FROM ticks WHERE code = '{}' and ts BETWEEN '{}' AND '{}' ".format(code,
                                                                                 date,
                                                                                 date+datetime.timedelta(days=1))
        df = pd.read_sql(sql, connection, parse_dates = ['ts'], index_col=['ts'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    
    if is_Futures: #若為期貨
            ticks = api.ticks(
                contract = api.Contracts.Futures.get(code)[codes],  # For futures, use the Futures contract
                date=date.strftime('%Y-%m-%d') 
                )

    else: #若為證券
            ticks = api.ticks(
            contract=api.Contracts.Stocks[code],  # For stocks, use the Stocks contract
                date=date.strftime('%Y-%m-%d')
            )    
    df = pd.DataFrame({**ticks})

    df.ts = pd.to_datetime(df.ts)
    df['code'] = code
    df = df.set_index('ts')

    return df, False

In [ ]:
df, in_db = get_ticks(connection, api, '2024/10/24', '2330')

print(in_db)
df

In [27]:
def update_ticks(connection, api, daily_target, is_Futures = False):
    main_df = pd.DataFrame()

    tw_calendar = get_calendar('XTAI')

    for date, codes in daily_target.items():

        day_trading_codes = [code for code in codes ]

        print('正在更新{}逐筆成交資料，總共{}檔標的，為{}'.format(date.strftime('%Y/%m/%d'), len(day_trading_codes),day_trading_codes))

        for code in day_trading_codes:
            df, in_db = get_ticks(connection, api, date, code, is_Futures)

            if df is not None and in_db:
                main_df = pd.concat([main_df, df], sort = False)
                time.sleep(1)

            prev_trading_date = tw_calendar.previous_close(date).date()
            prev_df, prev_in_db = get_ticks(connection, api, date, code, is_Futures)

            if prev_df is not None and not prev_in_db:
                main_df = pd.concat([main_df, prev_df], sort = False)
                time.sleep(1)
    
    if not main_df.empty:
        try:
            main_df.to_sql('ticks', connection, if_exists='append', index=False)
            print("Data stored successfully.")
        except Exception as e:
            print("Failed to store data:", e)
    return main_df



In [28]:
daily_targets ={
    pd.to_datetime('2024-08-22'): ['2317', '2330'],
    pd.to_datetime('2024-08-23'): ['2317', '2330'],
    pd.to_datetime('2024-08-24'): ['2317', '2330']
                }

In [29]:
connection = sqlite3.connect('work_data.db')
update_ticks(connection, api, daily_targets)

正在更新2024/08/22逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/23逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/24逐筆成交資料，總共2檔標的，為['2317', '2330']
Data stored successfully.


,tick_type,bid_price,bid_volume,ask_volume,volume,close,ask_price,code
ts,,,,,,,,
2024-08-22 09:00:00.362644,1.0,183.5,374.0,252.0,831.0,184.0,184.0,2317
2024-08-22 09:00:01.879958,1.0,183.5,400.0,305.0,1.0,184.0,184.0,2317
2024-08-22 09:00:01.967940,1.0,183.5,400.0,303.0,2.0,184.0,184.0,2317
2024-08-22 09:00:02.593794,1.0,183.5,401.0,302.0,1.0,184.0,184.0,2317
2024-08-22 09:00:04.008736,1.0,183.5,403.0,305.0,1.0,184.0,184.0,2317
...,...,...,...,...,...,...,...,...
2024-08-23 13:24:54.884657,1.0,949.0,62.0,102.0,1.0,950.0,950.0,2330
2024-08-23 13:24:55.499025,1.0,949.0,62.0,101.0,1.0,950.0,950.0,2330
2024-08-23 13:24:57.996492,2.0,949.0,63.0,102.0,1.0,949.0,950.0,2330
